<a href="https://colab.research.google.com/github/YashM246/Hamoye_Stage_B_ML_Regression/blob/main/Hamoye_Stage_B_ML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hamoye Stage B - Machine Learning: Regression

## Predicting Energy Efficiency of Buildings

## Importing Required Libraries



In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import io
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

## Importing Dataset

In [2]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['energydata_complete.csv']))
df.head()

Saving energydata_complete.csv to energydata_complete.csv


,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
column_names = {'T1':'Kitchen_Temperature', 'RH_1': 'Kitchen_Humidity', 
                'T2':  'Living_Room_Temperature', 'RH_2': 'Living_RoomHumidity', 'T3': 'Laundry_Room_Temperature',
                'RH_3': 'Laundry_Room_Humidity', 'T4': 'Office_Room_Temperature', 'RH_4': 'Office_Humidity',
                'T5': 'Bathroom_Temperature', 'RH_5': 'Bathroom_Humidity', 'T6':'Temperature_Outside_Building', 
                'RH_6': 'Humidity_Outside_Building','T7':'Ironing_Room_Temperature', 'RH_7':'Ironing_Room_Humidity', 
                'T8':'Teenager_Room_Temperature', 'RH_8':'Teenager_Room_Humidity', 'T9':'Parents_Room_Temperature', 'RH_9':
                'Parents_RoomHumidity', 'To':'Temperature_Outside', 'RH_out':'Humidity_Outside'}


df = df.rename(columns=column_names)
df.head()

,date,Appliances,lights,Kitchen_Temperature,Kitchen_Humidity,Living_Room_Temperature,Living_RoomHumidity,Laundry_Room_Temperature,Laundry_Room_Humidity,Office_Room_Temperature,...,Parents_Room_Temperature,Parents_RoomHumidity,T_out,Press_mm_hg,Humidity_Outside,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


## Question 12 <br>

### From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [15]:
x = df['Living_Room_Temperature'].values
y = df['Temperature_Outside_Building'].values
x = x.reshape(-1,1)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [17]:
score = r2_score(y_test, y_pred)
print(round(score,2))

0.64


## Question 13<br>
### Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:<br>What is the Mean Absolute Error (in two decimal places)?

In [18]:
df = df.drop(["date", "lights"], axis=1)

scaler = MinMaxScaler()

df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [19]:
x = df.drop(["Appliances"], axis=1)
y = df["Appliances"]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [21]:
model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [22]:
mae = mean_absolute_error(y_test, y_pred)
round(mae, 2)

0.05

## Question 14<br>
### What is the Residual Sum of Squares (in two decimal places)?

In [27]:
rss = np.sum(np.square(y_test - y_pred))
round(rss, 2)

45.35

## Question 15<br>
### What is the Root Mean Squared Error (in three decimal places)?

In [28]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
round(rmse, 3)

0.088

## Question 16
### What is the Coefficient of Determination (in two decimal places)?

In [29]:
score = r2_score(y_test, y_pred)
round(score,2)

0.15

## Question 17
### Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?







In [31]:
importance = model.coef_
highest_feature = ""
lowest_feature = ""
highest = 0
lowest = 0
feature_position = 0
highest_position = 0
lowest_position = 0
# summarize feature importance
for i,v in enumerate(importance):
    
    print('Feature: %0d, Score: %.5f' % (i,v))
    if (v > highest):
        highest = v
        highest_feature = i
        highest_position = feature_position
    if (v<lowest):
        lowest = v   
        lowest_feature = i
        lowest_position = feature_position
    feature_position = feature_position+1

Feature: 0, Score: -0.00328
Feature: 1, Score: 0.55355
Feature: 2, Score: -0.23618
Feature: 3, Score: -0.45670
Feature: 4, Score: 0.29063
Feature: 5, Score: 0.09605
Feature: 6, Score: 0.02898
Feature: 7, Score: 0.02639
Feature: 8, Score: -0.01566
Feature: 9, Score: 0.01601
Feature: 10, Score: 0.23642
Feature: 11, Score: 0.03805
Feature: 12, Score: 0.01032
Feature: 13, Score: -0.04461
Feature: 14, Score: 0.10200
Feature: 15, Score: -0.15760
Feature: 16, Score: -0.18994
Feature: 17, Score: -0.03980
Feature: 18, Score: -0.32186
Feature: 19, Score: 0.00684
Feature: 20, Score: -0.07767
Feature: 21, Score: 0.02918
Feature: 22, Score: 0.01231
Feature: 23, Score: 0.11776
Feature: 24, Score: 0.00077
Feature: 25, Score: 0.00077


In [33]:
new_df = df.drop(columns = ['Appliances'])

In [34]:
new_df.columns[highest_position]

'Kitchen_Humidity'

In [35]:
new_df.columns[lowest_position]

'Living_RoomHumidity'

Highest ---> Kitchen_Humidity i.e. RH_1<br>
Lowest ----> Living_RoomHumidity i.e. RH_2

## <br>Question 18
### Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [37]:
ridge = Ridge(alpha= 0.4)
ridge.fit(x_train, y_train)

Ridge(alpha=0.4)

In [38]:
ridge_pred = ridge.predict(x_test)

In [40]:
rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
round(rmse,3)

0.088

## Question 19
### Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [41]:
lasso = Lasso(alpha= 0.001)
lasso.fit(x_train, y_train)

Lasso(alpha=0.001)

In [43]:
importance = lasso.coef_
non_zero = 0
# summarize feature importance
for i,v in enumerate(importance):
    if (v !=0):
        non_zero = non_zero+1
print(non_zero)

4


## Question 20
### What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [44]:
lasso_pred = lasso.predict(x_test)

In [46]:
rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
round(rmse,3)

0.094